In [5]:
from kafka import KafkaConsumer, TopicPartition
import pandas as pd
import json
import time
from datetime import datetime, timedelta

kafkaBrokers='logging-kafka-kafka-kafkaroute-bootstrap-kafka.apps.cluster-7msqx.dynamic.redhatworkshops.io:443'
caRootLocation='/workspaces/openshift-lab/ai/training/ssl/CARoot.pem'
certLocation='/workspaces/openshift-lab/ai/training/ssl/certificate.pem'
keyLocation='/workspaces/openshift-lab/ai/training/ssl/key.pem'
password='c97xOn13878b'

# Capture my app logs into CSV
I am doing this for ease of data manipulation during model training. Once model train is done it will be real time query for 24hrs worth of log data

In [ ]:
import re

topic='ocplogs-myapp'
consumer = KafkaConsumer(topic,
    bootstrap_servers=[kafkaBrokers],
    value_deserializer=lambda x: x.decode('utf-8'),
    security_protocol='SSL',
    ssl_check_hostname=False,
    ssl_cafile=caRootLocation,
    ssl_certfile=certLocation,
    # ssl_keyfile=keyLocation,
    ssl_password=password,
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    consumer_timeout_ms=180000 # 3 minutes
)

timestamp_24hrs_ago = int((datetime.now() - timedelta(days=1)).timestamp() * 1000)

# Get the partitions for the topic
partitions = consumer.partitions_for_topic(topic)
topic_partitions = [TopicPartition(topic, p) for p in partitions]

# Get the offsets for the timestamp 24 hours ago
offsets = consumer.offsets_for_times({tp: timestamp_24hrs_ago for tp in topic_partitions})

# Seek to the calculated offsets
for tp in topic_partitions:
    if offsets[tp] is not None:
        consumer.seek(tp, offsets[tp].offset)

end_offsets = consumer.end_offsets(topic_partitions)

data = []
count=0
for event in consumer:
    tp = TopicPartition(event.topic, event.partition)
    if event.offset >= end_offsets[tp]:
        break
    if count>30000:
        break
    msg = event.value
    if msg is None:
        print("Waiting...")
    else:
        obj = json.loads(msg)
        timestamp = obj.get('@timestamp', 'N/A')
        namespace_name = obj.get('kubernetes', {}).get('namespace_name', 'N/A')
        app_name = obj.get('kubernetes', {}).get('container_name', 'N/A')
        message = obj.get('message', 'N/A')
        if pd.isna(message) or message.startswith('\t'):
            continue
        level = obj.get('level', 'N/A')
        log_type = obj.get('log_type', 'N/A')
        data.append({
            'timestamp': timestamp,
            'namespace_name': namespace_name,
            'app_name': app_name,
            'level': level,
            'log_type': log_type,
            'message': message,
            'target_classification': '',
        })
        # print(f"Namespace: {namespace_name}, app: {app_name}, Level: {level}, Type: {log_type}, Message: |{message}|")
        # if app_name != 'kafka':
        #     # print(f"level: {level}, count: {count}")
        #     count = count + 0 # do nothing
        count+=1

df = pd.DataFrame(data)
# now fill up target classification the best way possible to prepare training data

df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == "I am busybox. Running normally.." else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'invoice generated successfully' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'backup completed' else row['target_classification'], axis=1)


df['target_classification'] = df['message'].apply(lambda x: 'error' if x == "I am busybox. I have just faced an issue..." else '')
df['target_classification'] = df.apply(lambda row: 'error' if row['message'] == "RAID array experienced disk crash" else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Lead conversion failed for prospect ID \d{4} due to missing contact information\.", row['message']) else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Escalation rule execution failed for ticket ID \d{4} - undefined escalation level\.", row['message']) else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Customer follow-up process for lead ID \d{4} failed due to missing next action\.", row['message']) else row['target_classification'], axis=1)

df['target_classification'] = df.apply(lambda row: 'security alert' if 'suspicious activity observed from ip' in row['message'] else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'security alert' if row['message'] == 'security breach occurred. user tried to log in from outside of business hours' else row['target_classification'], axis=1)

df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == "The 'ExportToCSV' feature is outdated. Please migrate to 'ExportToXLSX' by the end of Q3." else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == 'Support for legacy authentication methods will be discontinued after 2025-06-01.' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == 'Support for legacy authentication methods will be discontinued after 2025-06-01.' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == "API endpoint 'getCustomerDetails' is deprecated and will be removed in version 3.2. Use 'fetchCustomerInfo' instead." else row['target_classification'], axis=1)


df.to_csv('myapp_logs.nogit.csv', index=False)

offset: 257 end_offset: 2222
offset: 557 end_offset: 2222
offset: 373 end_offset: 2282
offset: 673 end_offset: 2282
offset: 441 end_offset: 2258
offset: 775 end_offset: 2282
offset: 1075 end_offset: 2282
offset: 862 end_offset: 2222
offset: 1162 end_offset: 2222
offset: 6198 end_offset: 8020
offset: 5917 end_offset: 7901
offset: 6217 end_offset: 7901
offset: 6035 end_offset: 8062
offset: 6335 end_offset: 8062
offset: 6155 end_offset: 7896
offset: 1400 end_offset: 3366
offset: 1700 end_offset: 3366
offset: 1556 end_offset: 3431
offset: 1856 end_offset: 3431
offset: 1674 end_offset: 3526
offset: 755 end_offset: 2258
offset: 1055 end_offset: 2258
offset: 1375 end_offset: 2222
offset: 1675 end_offset: 2222
offset: 1492 end_offset: 2282
offset: 1909 end_offset: 3366
offset: 2209 end_offset: 3366
offset: 2065 end_offset: 3431
offset: 2365 end_offset: 3431
offset: 1463 end_offset: 2258
offset: 1783 end_offset: 2222
offset: 2083 end_offset: 2222
offset: 6567 end_offset: 7901
offset: 6867 end_o

In [ ]:
import re

# Load the CSV file into a DataFrame
df = pd.read_csv('/workspaces/openshift-lab/ai/model/myapp_logs.nogit.csv')

# Initialize the target_classification column with empty strings
df['target_classification'] = ''

# Apply the classification logic

df['target_classification'] = df['message'].apply(lambda x: 'info' if x == "I am busybox. Running normally.." else '')
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'invoice generated successfully' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'backup completed' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'application is up and running within acceptable parameters' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'info' if row['message'] == 'invoice generated successfully' else row['target_classification'], axis=1)

df['target_classification'] = df['message'].apply(lambda x: 'error' if x == "I am busybox. I have just faced an issue..." else '')
df['target_classification'] = df.apply(lambda row: 'error' if row['message'] == "RAID array experienced disk crash" else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Lead conversion failed for prospect ID \d{4} due to missing contact information\.", row['message']) else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Escalation rule execution failed for ticket ID \d{4} - undefined escalation level\.", row['message']) else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'error' if re.match(r"Customer follow-up process for lead ID \d{4} failed due to missing next action\.", row['message']) else row['target_classification'], axis=1)

df['target_classification'] = df.apply(lambda row: 'security alert' if 'suspicious activity observed from ip' in row['message'] else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'security alert' if row['message'] == 'security breach occurred. user tried to log in from outside of business hours' else row['target_classification'], axis=1)

df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == "The 'ExportToCSV' feature is outdated. Please migrate to 'ExportToXLSX' by the end of Q3." else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == 'Support for legacy authentication methods will be discontinued after 2025-06-01.' else row['target_classification'], axis=1)
df['target_classification'] = df.apply(lambda row: 'warning' if row['message'] == "API endpoint 'getCustomerDetails' is deprecated and will be removed in version 3.2. Use 'fetchCustomerInfo' instead." else row['target_classification'], axis=1)


# this is a hack. I have no idea why the "info" (as written above) block did not work 
# I inspected the data and looks like other that info every other row has taregt_classification fill out. 
# so I jumped the conclusion that the info block did not work and I am filling whatever is empty with info.
df['target_classification'] = df['target_classification'].apply(lambda x: 'info' if x == '' else x)

# Save the modified DataFrame back to a CSV file
df.to_csv('/workspaces/openshift-lab/ai/model/myapp_logs_with_classification.nogit.csv', index=False)

# Display the first few rows of the modified DataFrame
df.head()

,timestamp,namespace_name,app_name,level,log_type,message,target_classification
0,2025-03-09T04:53:12.143530588Z,myapp,busybox,default,application,I am busybox. Running normally..,info
1,2025-03-09T04:56:37.451478037Z,myapp,billing,default,application,security breach occurred. user tried to log in...,security alert
2,2025-03-09T04:56:50.646327206Z,myapp,legacycrm,default,application,The 'ExportToCSV' feature is outdated. Please ...,warning
3,2025-03-09T04:57:20.651643036Z,myapp,legacycrm,default,application,API endpoint 'getCustomerDetails' is deprecate...,warning
4,2025-03-09T04:57:47.467350244Z,myapp,billing,default,application,suspicious activity observed from ip 10.2.4.5,security alert


# Capture application logs into CSV

In [ ]:
topic='ocplogs-application'
consumer = KafkaConsumer(topic,
    bootstrap_servers=[kafkaBrokers],
    value_deserializer=lambda x: x.decode('utf-8'),
    security_protocol='SSL',
    ssl_check_hostname=False,
    ssl_cafile=caRootLocation,
    ssl_certfile=certLocation,
    # ssl_keyfile=keyLocation,
    ssl_password=password,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
)
data = []
count=0
for event in consumer:
    if count>8000:
        break
    msg = event.value
    if msg is None:
        print("Waiting...")
    else:
        obj = json.loads(msg)
        timestamp = obj.get('@timestamp', 'N/A')
        namespace_name = obj.get('kubernetes', {}).get('namespace_name', 'N/A')
        app_name = obj.get('kubernetes', {}).get('container_name', 'N/A')
        message = obj.get('message', 'N/A')
        if pd.isna(message) or message.startswith('\t'):
            continue
        level = obj.get('level', 'N/A')
        log_type = obj.get('log_type', 'N/A')
        data.append({
            'timestamp': timestamp,
            'namespace_name': namespace_name,
            'app_name': app_name,
            'level': level,
            'log_type': log_type,
            'message': message
        })
        # print(f"Namespace: {namespace_name}, app: {app_name}, Level: {level}, Type: {log_type}, Message: |{message}|")
        # if app_name != 'kafka':
        #     # print(f"level: {level}, count: {count}")
        #     count = count + 0 # do nothing
        count+=1

df = pd.DataFrame(data)
df.to_csv('application_logs.nogit.csv', index=False)

ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

# Capture audit logs in CSV

In [ ]:
topic='ocplogs-audit'
consumer = KafkaConsumer(topic,
    bootstrap_servers=[kafkaBrokers],
    value_deserializer=lambda x: x.decode('utf-8'),
    security_protocol='SSL',
    ssl_check_hostname=False,
    ssl_cafile=caRootLocation,
    ssl_certfile=certLocation,
    # ssl_keyfile=keyLocation,
    ssl_password=password,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
)

data = []
count=0
for event in consumer:
    if count>10000:
        break
    msg = event.value
    if msg is None:
        print("Waiting...")
    else:
        obj = json.loads(msg)
        timestamp = obj.get('@timestamp', 'N/A')
        kind = obj.get('kind', 'N/A')
        level = obj.get('level', 'N/A')
        source_ips = ', '.join(obj.get('sourceIPs', []))
        log_source = obj.get('log_source', 'N/A')
        log_type = obj.get('log_type', 'N/A')
        user_name = obj.get('user', {}).get('username', 'N/A')
        data.append({
            'timestamp': timestamp,
            'kind': kind,
            'level': level,
            'user_name': user_name,
            'log_type': log_type,
            'log_source': log_source,            
            'source_ips': source_ips,            
        })
        count+=1

df = pd.DataFrame(data)
df.to_csv('audit_logs.nogit.csv', index=False)

# capture infra logs into csv

In [ ]:
topic='ocplogs-infrastructure'
consumer = KafkaConsumer(topic,
    bootstrap_servers=[kafkaBrokers],
    value_deserializer=lambda x: x.decode('utf-8'),
    security_protocol='SSL',
    ssl_check_hostname=False,
    ssl_cafile=caRootLocation,
    ssl_certfile=certLocation,
    # ssl_keyfile=keyLocation,
    ssl_password=password,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
)

data = []
count=0
for event in consumer:
    if count>10000:
        break
    msg = event.value
    if msg is None:
        print("Waiting...")
    else:
        obj = json.loads(msg)
        print(obj)
        timestamp = obj.get('@timestamp', 'N/A')
        runtime_scope = obj.get('_RUNTIME_SCOPE', 'N/A')
        level = obj.get('level', 'N/A')
        log_source = obj.get('log_source', 'N/A')
        log_type = obj.get('log_type', 'N/A')
        tag = obj.get('tag', 'N/A')
        message = obj.get('message', 'N/A')
        data.append({
            'timestamp': timestamp,
            'runtime_scope': runtime_scope,
            'level': level,
            'log_type': log_type,
            'log_source': log_source,
            'tag': tag,
            'message': message                  
        })
        count+=1

df = pd.DataFrame(data)
df.to_csv('infra_logs.nogit.csv', index=False)